In [3]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from requests import get
from urllib.request import urlopen
import re
from collections import defaultdict
import time
import random
from datetime import datetime
import yagmail

### Crypto alert

**Kraken crypto list**

to-do:
- Have to solve email issue
- Adatbázist építeni 20 percenként frissülő adatokkal - kb. kész
    - még price diff-hez kell rakni crypto neveket
- email notification-okat beépíteni (email címet valahogy el kell rejteni)
    - 20 perces és 1 órás és 1 napos warning + nap végén growth recap
- eldönteni hogy deployoljuk (django, heroku?)
  - Ha heroku, akkor mehet egy app is hozzá ábrákkal
  
 https://aws.amazon.com/free/?all-free-tier.sort-by=item.additionalFields.SortRank&all-free-tier.sort-order=asc&awsf.Free%20Tier%20Categories=categories%23compute&trk=ps_a134p000006paeIAAQ&trkCampaign=acq_paid_search_brand&sc_channel=PS&sc_campaign=acquisition_EEM&sc_publisher=Google&sc_category=Cloud%20Computing&sc_country=EEM&sc_geo=EMEA&sc_outcome=acq&sc_detail=%2Bfree%20%2Bec2&sc_content=EC2%20Pricing_bmm&sc_matchtype=b&sc_segment=495055137481&sc_medium=ACQ-P|PS-GO|Brand|Desktop|SU|Cloud%20Computing|EC2|EEM|EN|Text|xx|EU&s_kwcid=AL!4422!3!495055137481!b!!g!!%2Bfree%20%2Bec2&ef_id=CjwKCAjwjuqDBhAGEiwAdX2cj1IplIhptO27cYwhzeSXJPRILD6epOWFir-XxjcFrybDjI5wBiZKlhoCGqUQAvD_BwE:G:s&s_kwcid=AL!4422!3!495055137481!b!!g!!%2Bfree%20%2Bec2&awsf.Free%20Tier%20Types=*all

In [4]:
headers = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})

**PANDAS version**

In [5]:
title_list = []
length = len(BeautifulSoup(
            get('https://coinranking.com/exchange/2Vu9j1PmUbx+kraken/coins', 
            headers=headers).text, 
            'html.parser').find_all('td',class_="table__cell table__cell--3-of-8 table__cell--s-6-of-10"))
for crypto in range(length):
    crypto_name = str(BeautifulSoup(
            get('https://coinranking.com/exchange/2Vu9j1PmUbx+kraken/coins', 
                headers=headers).text, 
                'html.parser').find_all('td',class_="table__cell table__cell--3-of-8 table__cell--s-6-of-10")[crypto].find_all(
                href=True)[0]).split("\n")[1].replace(" ", "")
    title_list.append(crypto_name)

In [6]:
price_data = pd.DataFrame()
counter_1 = 0 #counter for 60 minute allerts
counter_12 = 0

user = ''
app_password = '' # a token for gmail
to = ''

In [ ]:
while True: 
    if datetime.now().time().hour != 0:    
        av_cryptos = []
        price_list = []
        counter_1 += 1
        counter_12 += 1

        for item in range(len(title_list)):

            try: 

                price = float(
                    BeautifulSoup(
                        get('https://coinmarketcap.com/currencies/{}/'.format(title_list[item])).text, 
                            'html.parser').find_all(
                            "div", 
                            class_='priceValue___11gHJ')[0].text.replace(
                            '$','').replace(",", ""))

                av_cryptos.append(title_list[item])
                price_list.append(price)
                #print(price)

                time.sleep(6)

            except IndexError:
                #print(title_list[item])
                continue

        now = datetime.now().time()
        price_data['Crypto names'] = av_cryptos
        price_data[str(now.hour) + ':' + str(now.minute)] = price_list
        
        
        #20 minute allert
        if len(price_data.columns) >= 3:
            
            price_diff = dict(zip(price_data['Crypto names'].to_list(), 
                                  list(price_data[price_data.columns.to_list()[-1]]/
                                       price_data[price_data.columns.to_list()[-2]])))
            

            big_price_inc = {}
            
            for name,value in price_diff.items():
                if value > 1.05:
                    big_price_inc[name] = value
            
            if len(big_price_inc) > 0:
                
                subject = '20 minutes alert {}'.format(str(datetime.now()))
                content = ["""List of cryptos with more than 5%-point 
                              price increase in the last 20 minutes: """, 
                              str(big_price_inc)]

                with yagmail.SMTP(user, app_password) as yag:
                    yag.send(to, subject, content)
        else:
            pass

        #60 minute allert
        if counter_1 == 4:
            
            price_diff_60 = dict(zip(price_data['Crypto names'].to_list(), 
                                  list(price_data[price_data.columns.to_list()[-1]]/
                                       price_data[price_data.columns.to_list()[-4]])))
            
            big_price_inc_60 = {}
            
            for name,value in price_diff_60.items():
                #have to set correct rate
                if value > 1.15:
                    big_price_inc_60[name] = value
                    
            counter_1 = 0
            
            if len(big_price_inc_60) > 0:
                
                subject = '60 minutes alert {}'.format(str(datetime.now()))
                content = ["""List of cryptos with more than 15%-point 
                              price increase in the last 60 minutes: """, 
                              str(big_price_inc_60)]

                with yagmail.SMTP(user, app_password) as yag:
                    yag.send(to, subject, content)
        
        else:
            pass

        #12 hour allert
        if counter_12 == 37:
            
            price_diff_12hour = dict(zip(price_data['Crypto names'].to_list(), 
                                  list(price_data[price_data.columns.to_list()[-1]]/
                                       price_data[price_data.columns.to_list()[-36]])))
            
            big_price_inc_12hour = {}
            
            for name,value in price_diff_12hour.items():
                #have to set correct rate
                if value > 1.2:
                    big_price_inc_12hour[name] = value
                    
            counter_12 = 0
            
            if len(big_price_inc_12hour) > 0:
                
                subject = '12 hour alert {}'.format(str(datetime.now()))
                content = ["""List of cryptos with more than 20%-point 
                              price increase in the last 12 hours: """, 
                              str(big_price_inc_12hour)]

                with yagmail.SMTP(user, app_password) as yag:
                    yag.send(to, subject, content)
                    
        else:
            pass

        #20 minutes sleep (16 min + 4 min runtime) - 960 will be 20 min
        time.sleep(960)


    else:
        
        price_diff_daily = dict(zip(price_data['Crypto names'].to_list(), 
                                  list(price_data[price_data.columns.to_list()[-1]]/
                                       price_data[price_data.columns.to_list()[1]])))
        
        subject = 'Daily recap {}'.format(str(datetime.now()))
        content = ["""Daily crypto price changes: """, 
                      str(price_diff_daily)]

        with yagmail.SMTP(user, app_password) as yag:
            yag.send(to, subject, content)
                    
        print("reset time")
        time.sleep(3600)
        #save data (maybe to big dataframe? - maybe less important)
        price_data = pd.DataFrame()
        
        title_list = []
        length = len(BeautifulSoup(
                    get('https://coinranking.com/exchange/2Vu9j1PmUbx+kraken/coins', 
                    headers=headers).text, 
                    'html.parser').find_all('td',class_="table__cell table__cell--3-of-8 table__cell--s-6-of-10"))
        for crypto in range(length):
            crypto_name = str(BeautifulSoup(
                    get('https://coinranking.com/exchange/2Vu9j1PmUbx+kraken/coins', 
                        headers=headers).text, 
                        'html.parser').find_all(
                        'td',class_="table__cell table__cell--3-of-8 table__cell--s-6-of-10")[crypto].find_all(
                        href=True)[0]).split("\n")[1].replace(" ", "")
            
            title_list.append(crypto_name)

**OLD - codes usefull for data preservation in gsheet (e.g end of each day)**

In [ ]:
#alter coin name source (inferior)
import cloudscraper

scraper = cloudscraper.create_scraper()  # returns a CloudScraper instance

title_list = []
length = len(BeautifulSoup(scraper.get(
    "https://www.kraken.com/learn/types-of-cryptocurrency").text, 
    'html.parser').find_all(class_="asset-list"))

for row in range(length):
    
    crypto_list = len(BeautifulSoup(
        scraper.get(
        "https://www.kraken.com/learn/types-of-cryptocurrency").text, 
        'html.parser').find_all(
        class_="asset-list")[row].find_all("a"))
    
    for crypto in range(crypto_list):
        
        crypto_name = BeautifulSoup(
            scraper.get(
            "https://www.kraken.com/learn/types-of-cryptocurrency").text, 
            'html.parser').find_all(
            class_="asset-list")[row].find_all(
            "a")[crypto].string.split(u'\xa0')[0]
        
        title_list.append(crypto_name)


In [ ]:
#data = urlopen("http://www.nasdaqtrader.com/dynamic/symdir/nasdaqlisted.txt") - később

In [ ]:
# titles = BeautifulSoup(
#     get('https://coinmarketcap.com/exchanges/kraken/', 
#         headers=headers).text, 
#         'html.parser').find_all(
#         'table')[2].find_all(
#         'a', class_="cmc-link")

# title_list = []
# for elem in range(len(titles)):
#     try:
#         if titles[elem]['title'] not in title_list:
#             title = titles[elem]['title'].replace(' ', '-')
#             title_list.append(title)
#     except KeyError:
#         pass
    
# title_list = list(set(title_list))

In [ ]:
# define the scope
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']

# add credentials to the account
creds = ServiceAccountCredentials.from_json_keyfile_name('credentials.json', scope)

# authorize the clientsheet 
client = gspread.authorize(creds)

In [ ]:
sheet = client.open("CrpytoPrices").sheet1

In [ ]:
av_cryptos = []
price_list = []


sheet.update('A1', 'Crypto names')


for item in range(len(title_list)):
    
    try: 
        
        price = float(
            BeautifulSoup(
                get('https://coinmarketcap.com/currencies/{}/'.format(title_list[item])).text, 
                    'html.parser').find_all(
                    "div", 
                    class_='priceValue___11gHJ')[0].text.replace(
                    '$','').replace(",", ""))
        
        av_cryptos.append(title_list[item])
        price_list.append(price)
          
    except IndexError:
        pass
    
cell_list_names = sheet.range('A2:A{}'.format(str(len(av_cryptos) + 1)))

for i, val in enumerate(av_cryptos):
    cell_list_names[i].value = val
    
sheet.update_cells(cell_list_names)
    
for row in range(len(av_cryptos)):
    
    #Column title
    now = datetime.now().time()
    column_header = str(now.hour) + ':' + str(now.minute)
    sheet.update('B1', column_header)
    
    #Get price data and cell range to copy to
    cell_list = sheet.range('B2:B{}'.format(str(len(av_cryptos) + 1)))
    
    for i, val in enumerate(price_list):
        cell_list[i].value = val
    
    sheet.update_cells(cell_list)